<a href="https://colab.research.google.com/github/SohamRatnaparkhi/BrainTumorDetection/blob/main/Brain_tumor_big_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image
import os

# Data prepration

In [ ]:
!unzip "/content/brain_tumor_dataset.zip"

Declaring constants

In [3]:
SIZE = 224

path = r"/content/brain_tumor_dataset"
yes_tumor_path = path + r"/yes"
no_tumor_path = path + r"/no"
yes_tumor = os.listdir(yes_tumor_path)
no_tumor = os.listdir(no_tumor_path)

len(os.listdir(no_tumor_path)), len(os.listdir(yes_tumor_path))

(1500, 1500)

Resizing all images in `224x224` pixel format

In [4]:
# Resize yes_tumor images
for p in yes_tumor:
    path = yes_tumor_path+"/"+p
    # resize_save(path)
    try:
        im = Image.open(path)
        im = im.resize((SIZE, SIZE))
        im.save(path)
    except:
        print(path)
        break
# Resize no_tumor
for p in no_tumor:
    path = no_tumor_path+"/"+p
    # resize_save(path)
    try:
        im = Image.open(path)
        im = im.resize((SIZE, SIZE))
        im.save(path)
    except:
        print(path)
        break

In [5]:
len(os.listdir(yes_tumor_path)), len(os.listdir(no_tumor_path))

(1500, 1500)

Creating `data` - `label` pairs
> yes_tumor -> 1
>>
> no_tumor -> 0

In [6]:
data = []
label = []

for p in yes_tumor:
  path = yes_tumor_path+"/"+p
  image = cv.imread(path)
  # if image == None:
  #   print(p)
  data.append(image)
  label.append(1)

for p in no_tumor:
  path = no_tumor_path+"/"+p
  image = cv.imread(path)
  # if image == None:
  #   print(p)
  data.append(image)
  label.append(0)

len(data) == len(label) == 3000

True

Normalizing data

In [7]:
data = np.array(data)/255.0
label = np.array(label)

data.shape, label.shape

((3000, 224, 224, 3), (3000,))

Splitting the data in `train` and `test` in **80-20** ratio

In [8]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(data, label, test_size = 0.20)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2400, 224, 224, 3), (600, 224, 224, 3), (2400,), (600,))

# Training the model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [11]:
model_1 = Sequential()

model_1.add(Conv2D(64, kernel_size=3, activation='relu'))
model_1.add(MaxPooling2D(pool_size = (2,2)))

model_1.add(Conv2D(64, kernel_size=3, activation='relu', kernel_initializer="he_uniform"))
model_1.add(Conv2D(64, kernel_size=3, activation='relu'))

model_1.add(Conv2D(64, kernel_size=3, activation='relu', kernel_initializer="he_uniform"))
model_1.add(Conv2D(64, kernel_size=3, activation='relu'))

model_1.add(Flatten())

model_1.add(Dense(1, activation='sigmoid'))


model_1.compile(optimizer=Adam(), loss = BinaryCrossentropy(),  metrics = ["accuracy"])


history = model_1.fit(x_train, y_train, epochs = 10, validation_data=(x_test, y_test), batch_size = 16)


model_1.save("Big_Brain_tumor_model_1.h5")

Epoch 1/10
150/150 [==============================] - 26s 80ms/step - loss: 0.5453 - accuracy: 0.7550 - val_loss: 0.4069 - val_accuracy: 0.7867
Epoch 2/10
150/150 [==============================] - 11s 70ms/step - loss: 0.2916 - accuracy: 0.8821 - val_loss: 0.1670 - val_accuracy: 0.9367
Epoch 3/10
150/150 [==============================] - 10s 70ms/step - loss: 0.1155 - accuracy: 0.9613 - val_loss: 0.2459 - val_accuracy: 0.9017
Epoch 4/10
150/150 [==============================] - 10s 70ms/step - loss: 0.0745 - accuracy: 0.9779 - val_loss: 0.1221 - val_accuracy: 0.9617
Epoch 5/10
150/150 [==============================] - 11s 70ms/step - loss: 0.0264 - accuracy: 0.9908 - val_loss: 0.1859 - val_accuracy: 0.9683
Epoch 6/10
150/150 [==============================] - 11s 71ms/step - loss: 0.0204 - accuracy: 0.9950 - val_loss: 0.1677 - val_accuracy: 0.9683
Epoch 7/10
150/150 [==============================] - 11s 71ms/step - loss: 0.0208 - accuracy: 0.9925 - val_loss: 0.2033 - val_accuracy:

In [13]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 107, 107, 64)      36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 105, 105, 64)      36928     
                                                                 
 conv2d_4 (Conv2D)           (None, 103, 103, 64)      36928     
                                                        

Generating reports

In [ ]:
from sklearn import preprocessing as p
from sklearn.metrics import classification_report, confusion_matrix

predictions = model_1.predict(x_test, batch_size= 16)
predictions = np.argmax(predictions, axis= 1)
actuals = np.argmax(y_test, axis= 0)

print(classification_report(actuals, predictions, target_names= p.label_binarizer.classes_))

cm = confusion_matrix(actuals, predictions)
print(cm)